UPDATE: No nee to install matrixprofile, as the stumpy module has everything we need!!

To install matrixprofile, run in order the following cells:

Look for what version of python is here. If >3.9, run the following cells to downgrade it.

In [ ]:
!python --version

Python 3.10.12


Look for where python is installed to change the path in the "!ln -sf /usr/local/bin/python3.8 /usr/local/bin/python" in the following cell. (Add or remove "local/" if encountering issues)

In [ ]:
!which python

/usr/local/bin/python


In [ ]:
!sudo apt-get install python3.8
!python3.8 --version
# !ln -sf /usr/local/bin/python3.8 /usr/local/bin/python
!ln -sf /usr/bin/python3.8 /usr/bin/python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support
  python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support python3.8
  python3.8-minimal
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,098 kB of archives.
After this operation, 18.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.8-minimal amd64 3.8.18-1+jammy1 [794 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.8-minima

In [ ]:
!sudo apt-get install python3.8-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3.8-lib2to3
The following NEW packages will be installed:
  python3.8-distutils python3.8-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,237 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.8-lib2to3 all 3.8.18-1+jammy1 [126 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.8-distutils all 3.8.18-1+jammy1 [193 kB]
Fetched 319 kB in 1s (419 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize fronte

Check if the version is updated from "Python 3.10" to "Python 3.8".

In [ ]:
!python --version

Python 3.10.12


Install matrixprofile, then restart the session.

In [ ]:
!python -m pip install matrixprofile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.6/331.6 kB 6.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
## Eventually, if the previous cell isn't working, try this one
# !rm matrixprofile
# !git clone https://github.com/matrix-profile-foundation/matrixprofile.git
# %cd matrixprofile
# !pip install .

Check if the installation was successful (maybe try restarting the session a few times).

In [ ]:
import matrixprofile as mp

ModuleNotFoundError: No module named 'matrixprofile'

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install stumpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
import pdb

import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_array

# Additional imports to deal with Matrix Profiling. Only needed for the matrix profiling part of the project that is currently commented out
# import matrixprofile as mp
import stumpy
from stumpy import stump

In [ ]:
# Mounting my drive to have access to the other files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Creating a dictionary of lists of paths to the correlation matrices for each method. Each list in the dictionary represents a different method.
methods = ['pearson', 'spearman', 'kendall', 'partial']
full_corr_path_lists = {}
for method in methods:
    method_dir = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/corr_matrices/corr_matrix_{method}/'
    full_corr_path_lists[method] = []
    for file in os.listdir(method_dir):
        full_corr_path_lists[method].append(file)

In [ ]:
# Generating the diagnostic file from the diagnostic_label.csv file
diagnostic_label = np.loadtxt('/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/diagnostic_label.csv', dtype=str, delimiter=',')

# Combining the 'EMCI', 'LMCI' and 'MCI' diagnostics into a single 'MCI' label for simplicity, then one-hot encoding the diagnostics
for patient in range(len(diagnostic_label)):
    if diagnostic_label[patient] == 'CN':
        diagnostic_label[patient] = 0
    elif diagnostic_label[patient] == 'SMC':
        diagnostic_label[patient] = 1
    elif diagnostic_label[patient] == 'EMCI' or diagnostic_label[patient] == 'LMCI' or diagnostic_label[patient] == 'MCI':
        diagnostic_label[patient] = 2
    elif diagnostic_label[patient] == 'AD':
        diagnostic_label[patient] = 3
    else:
        print('Error: Diagnostic label not recognised')
        break

In [ ]:
# Loading the age feature of patients to use as a node feature
ages = np.loadtxt('/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/age.csv', delimiter=',')
min_age = np.min(ages)
max_age = np.max(ages)

In [ ]:
# Prepocessing the sex feature of patients to use as a node feature. Here, 0 represents male patients and 1 represents female patients
sex = np.loadtxt('/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/sex.csv', dtype=str, delimiter=',')
for patient in range(len(sex)):
    if sex[patient] == 'M':
        sex[patient] = 0
    else:
        sex[patient] = 1

In [ ]:
# Defining functions to simplify the code in the class Raw_to_Graph.

# To convert a dictionnary into a numpy array
def dict_to_array(dict):
    array = np.array(list(dict.values()))
    return array

# To normalize an array
def normalize_array(array):
    norm_array = (array - np.mean(array)) / np.std(array)
    return norm_array

In [ ]:
# Functions to implement the matrix profile algorithm using STUMPY++.
# This is commented out as my local kernel dies when running this cell because of memory issues with matrix profiling. I acquired the data by running it on colab and saving it in the repo here. Only uncomment if needed.

# Computing the matrix profile using STUMPY++
def compute_matrix_profile(time_series, m=3):
    # Look into 3, 10, 20 m
    # Here the m parameter is the window size, i.e. the length of the subsequence to compute the matrix profile for.
    # Are we looking for short-lived neural activations (then small m), or are we interested in detecting longer-term changes or anomalies in brain activity (large m).
    # pdb.set_trace()
    matrix_profile = stump(time_series, m)
    return matrix_profile

# Finding the motif and discord in the matrix profile
def find_motif_discord(matrix_profile):
    motif_idx = motif_idx = np.argsort(matrix_profile[:, 0])[0]
    discord_idx = discord_idx = np.argsort(matrix_profile[:, 0])[-1]
    return motif_idx, discord_idx

# Function to compute the matrix profile, motifs and discords for a correlation matrix
def matrix_profile(corr_matrix):
    matrix_profiles = []
    motifs = []
    discords = []
    for row in corr_matrix:
        mp = compute_matrix_profile(row)
        matrix_profiles.append(mp)
        motif_idx, discord_idx = find_motif_discord(mp)
        motifs.append(motif_idx)
        discords.append(discord_idx)
    motifs_array = np.array(motifs)
    discords_array = np.array(discords)
    return matrix_profiles, motifs_array, discords_array

# Computing the matrix profiles for all the correlation matrices we have and saving them in a folder
for method in methods:
    method_dir = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/corr_matrices/corr_matrix_{method}/'
    for file in os.listdir(method_dir):
        corr_matrix = np.loadtxt(method_dir + file, delimiter=',')
        matrix_profiles, motifs, discords = matrix_profile(corr_matrix)
        path_mp = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/matrix_profiles/matrix_profile_{method}/'
        if not os.path.exists(path_mp):
          os.mkdir(path_mp)
        profile_dict = {}
        profile_dict['mp'] = matrix_profiles
        profile_dict['motifs'] = motifs
        profile_dict['discords'] = discords
        with open(f'{path_mp}/{file}', "wb") as fl:
          pkl.dump(profile_dict, fl)
        print("Done writing dict into .txt file")

Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writi

/usr/local/lib/python3.10/dist-packages/stumpy/core.py:3453: UserWarning: A large number of values in `P` are smaller than 1e-06.
For a self-join, try setting `ignore_trivial=True`.
  warnings.warn(msg)


Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writi

/usr/local/lib/python3.10/dist-packages/stumpy/core.py:2243: RuntimeWarning: divide by zero encountered in divide
  Σ_T_inverse = 1.0 / Σ_T


Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writing dict into .txt file
Done writi

In [ ]:
with open('/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/matrix_profiles/matrix_profile_pearson/patient_002_S_0295.csv', "rb") as fl:
  patient_dict = pkl.load(fl)

In [ ]:
np.array(patient_dict['mp']).shape

(116, 114, 4)

In [ ]:
# Defining a class to preprocess raw data into a format suitable for training Graph Neural Networks (GNNs).
## With the possibility of assigning weight to edges, adding the age feature, sex feature, and matrixe profiling.

class Raw_to_Graph(InMemoryDataset):
    def __init__(self, root, threshold, method, weight=False, age=False, sex=False, matrixprofile=False, transform=None, pre_transform=None):
        self.threshold = threshold
        self.method = method
        self.weight = weight
        self.age = age
        self.sex = sex
        self.matrixprofile = matrixprofile
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return ['data.pt']

    # This function is used to process the raw data into a format suitable for GNNs, by constructing graphs out of the connectivity matrices.
    def process(self):
        graphs=[]
        corr_matrices = full_corr_path_lists[self.method]
        for patient_idx, patient_matrix in enumerate(corr_matrices):
            path = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/corr_matrices/corr_matrix_{self.method}/{patient_matrix}'
            corr_matrix = pd.read_csv(path, header=None).values
            # Here ROIs stands for Regions of Interest
            nbr_ROIs = corr_matrix.shape[0]
            edge_matrix = np.zeros((nbr_ROIs,nbr_ROIs))
            for j in range(nbr_ROIs):
                for k in range(nbr_ROIs):
                    # Here we are using the absolute value of each element of the correlation matrix, as the corr coeff is in the range [-1,1].
                    if np.abs(corr_matrix[j,k]) < self.threshold:
                        edge_matrix[j,k] = 0
                    else:
                        if self.weight:
                            # Here we assign the absolute value of the correlation coefficient as the edge weight.
                            edge_matrix[j,k] = corr_matrix[j,k]
                        else:
                            # Here we assign 1 as the edge weight, i.e. regardless of the the absolute value of the correlation coefficient.
                            edge_matrix[j,k] = 1
            # Create a NetworkX graph from the edge matrix
            NetworkX_graph = from_numpy_array(edge_matrix)

            # Compute the degree, betweenness centrality, clustering coefficient, local efficiency for each node of the graph and the global efficiency of the graph
            degree_dict = dict(NetworkX_graph.degree())
            between_central_dict = nx.betweenness_centrality(NetworkX_graph)
            cluster_coeff_dict = nx.clustering(NetworkX_graph)
            global_eff = nx.global_efficiency(NetworkX_graph)
            local_eff_dict = {}
            for node in NetworkX_graph.nodes():
                subgraph_neighb = NetworkX_graph.subgraph(NetworkX_graph.neighbors(node))
                if subgraph_neighb.number_of_nodes() > 1:
                    efficiency = nx.global_efficiency(subgraph_neighb)
                else:
                    efficiency = 0.0
                local_eff_dict[node] = efficiency
            pdb.set_trace()

            # Convert the degree, betweenness centrality, local efficiency, clustering coefficient and ratio of local to global efficiency dictionaries to NumPy arrays then normalize them
            degree_array = dict_to_array(degree_dict)
            degree_array_norm = normalize_array(degree_array)

            between_central_array = dict_to_array(between_central_dict)
            between_central_array_norm = normalize_array(between_central_array)

            local_efficiency_array = dict_to_array(local_eff_dict)
            local_eff_array_norm = normalize_array(local_efficiency_array)

            ratio_local_global_array = dict_to_array(local_eff_dict) / global_eff
            ratio_local_global_array_norm = normalize_array(ratio_local_global_array)

            cluster_coeff_array = dict_to_array(cluster_coeff_dict)
            cluster_coeff_array_norm = normalize_array(cluster_coeff_array)

            # Initializing an array for the graph features
            x_array = np.stack([degree_array_norm, between_central_array_norm, local_eff_array_norm, cluster_coeff_array_norm, ratio_local_global_array_norm], axis=-1)
            x_array = x_array.astype(np.float32)
            # pdb.set_trace()

            if self.age:
                # Extracting the age feature of the patient
                patient_age = ages[patient_idx]
                age_norm = (patient_age - min_age) / (max_age - min_age)
                # Making the age array the same size as the other arrays
                age_array = np.full((nbr_ROIs,), age_norm)
                x_array = np.concatenate((x_array, age_array), axis=-1)
            if self.sex:
                # Extracting the sex feature of the patient
                patient_sex = int(sex[patient_idx])
                # Making the sex array the same size as the other arrays
                sex_array = np.full((nbr_ROIs,), patient_sex)
                x_array = np.concatenate((x_array, sex_array), axis=-1)

            if self.matrixprofile:
                print('Yes!')
                path = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/ADNI_full/matrix_profiles/matrix_profile_{method}/{patient_matrix}'
                with open(path, "rb") as fl:
                  patient_dict = pkl.load(fl)
                # combine dimensions
                features = np.array(patient_dict['mp']).reshape(len(patient_dict['mp']),-1)
                features = features.astype(np.float32)
                print('YES!')
                x_array = np.concatenate((x_array, features), axis=-1)
                print(x_array.shape)


            # Concatenate the degree, participation coefficient, betweenness centrality, local efficiency, and ratio of local to global efficiency arrays to form a single feature vector
            x = torch.tensor(x_array, dtype=torch.float)
            print(x.shape)

            # Create a Pytorch Geometric Data object from the NetworkX
            graph_data = from_networkx(NetworkX_graph)
            ## The feature matrix of the graph is the degree, betweenness centrality, local efficiency, clustering coefficient and ratio of local to global efficiency of each node
            graph_data.x = x
            ## The target/output variable that we want to predict is the diagnostic label of the patient
            graph_data.y = float(diagnostic_label[patient_idx])
            graphs.append(graph_data)

        data, slices = self.collate(graphs)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
# Defining a function to display some statistics and features about the dataset.
def dataset_features_and_stats(dataset):
    print()
    print(f'Dataset: {dataset}:')
    print('====================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Weighted: {dataset.weight}')
    print(f'Threshold: {dataset.threshold}')
    print(f'Correlation Method: {dataset.method}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {len(np.unique(diagnostic_label))}')

    # Getting the first graph object in the dataset.
    data = dataset[0]

    print()
    print(data)
    print('=============================================================')

    # Some statistics about the first graph.
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

In [ ]:
# Testing the class Raw_to_Graph with one example and saving it
threshold = 0.4
weight = False
age = False
sex = False
matrixprofile = True
method = 'pearson'

root = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/Raw_to_graph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MP{matrixprofile}'
dataset = Raw_to_Graph(root=root, threshold=threshold, method=method, weight=weight, age=age, sex=sex, matrixprofile=matrixprofile)
dataset_features_and_stats(dataset)


Dataset: Raw_to_Graph(197):
Number of graphs: 197
Weighted: False
Threshold: 0.4
Correlation Method: pearson
Number of features: 461
Number of classes: 4

Data(edge_index=[2, 1410], weight=[1410], x=[116, 461], y=[1], num_nodes=116)
Number of nodes: 116
Number of edges: 1410
Average node degree: 12.16
Has isolated nodes: True
Has self-loops: True
Is undirected: True


In [ ]:
# Testing the class Raw_to_Graph with different thresholds, weights and methods and saving it
# thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
thresholds = [0.4]
weights = [True, False]
age = False
sex = False
matrixprofile = True
# methods = ['pearson', 'spearman', 'kendall', 'partial']
methods = ['pearson']

for weight in weights:
    for method in methods:
        for threshold in thresholds:
            print(matrixprofile)
            root = f'/content/drive/MyDrive/Bachelor-Thesis/Bachelor-Thesis-Alzheimers-HGNN/Raw_to_graph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MP{matrixprofile}'
            dataset = Raw_to_Graph(root=root, threshold=threshold, method=method, weight=weight)
            dataset_features_and_stats(dataset)

Processing...


True
> <ipython-input-19-20840d778732>(60)process()
     58 
     59             # Convert the degree, betweenness centrality, local efficiency, clustering coefficient and ratio of local to global efficiency dictionaries to NumPy arrays then normalize them
---> 60             degree_array = dict_to_array(degree_dict)
     61             degree_array_norm = normalize_array(degree_array)
     62 

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
torch.Size([116, 5])
> <ipython-input-19-20840d778732>(60)process()
     58 
     59             # Convert the degree, betweenness centrality, local efficiency, clustering coefficient and ratio of local to global efficiency dictionaries to NumPy arrays then normalize them
---> 60             degree_array = dict_to_array(degree_dict)
     61             degree_array_norm = normalize_array(degree_array)
     62 

ipdb> q
